In [1]:
%run dataset.ipynb

In [2]:
class Office31Dataset(Dataset):
    @property     #'@property' 데코레이터 때문에 base 함수는 인수 없이 호출되는 속성 함수다.
    def base(self):
        return super(Office31Dataset, self)

In [3]:
#객체 초기화 함수 정의
def office31_init(self, resolution=[100,100], input_shape=[-1]):
    self.base.__init__('office31', 'dual_select')       #모드 값으로 dual_select로 지정한다.
    
    path = 'office31'
    domain_names = list_dir(path)
    
    images = []
    didxs, oidxs = [], []
    
    #도메인과 품폭이라는 두 차원으로 배치된 디렉터리 구조 때문에 반복문 처리가 삼중 구조이고 처리도 복잡하다.
    for dx, dname in enumerate(domain_names):
        domainpath = os.path.join(path, dname, 'images')
        object_names = list_dir(domainpath)
        for ox, oname in enumerate(object_names):
            objectpath = os.path.join(domainpath, oname)
            filenames = list_dir(objectpath)
            for fname in filenames:
                if fname[-4:] != '.jpg':   #office31 디렉터리를 순회하면서 'jpg'확장자의 이미지 파일들을 읽는다.
                    continue
                imagepath = os.path.join(objectpath, fname)
                pixels = load_image_pixels(imagepath, resolution, input_shape)    #resolution 매개변수는 이미지 데이터의 해상도를 지정
                images.append(pixels)
                didxs.append(dx)
                oidxs.append(ox)

    self.image_shape = resolution + [3]    #resolution에 칼라 이미지 RGB 성분을 나타내는 3을 추가해 3차원 형태로 self.image_shape값을 지정

    xs = np.asarray(images, np.float32)

    #onehot()을 이용하여 두 리스트에 담긴 번호들을 길이 3의 원-핫 벡터와 길이 31의 원-핫 벡터들을 갖는 ys0와 ys1로 변환한다.
    ys0 = onehot(didxs, len(domain_names))
    ys1 = onehot(oidxs, len(object_names))
    ys = np.hstack([ys0, ys1])           #np.hstack()를 이용해 둘을 연접시켜 길이 34의 벡터들로 구성된 정답 정보 ys를 만든다.
    
    self.shuffle_data(xs, ys, 0.8)
    self.target_names = [domain_names, object_names]    
    self.cnts = [len(domain_names)]    #self.cnts를 도메인 수에 해당하는 값을 원소로 하는 리스트 [3]으로 지정하게 된다.

In [4]:
#복합 출력 처리를 위한 순전파 함수 정의
def office31_forward_postproc(self, output, y):     #output은 34개의 열로 구성된 행렬 형태로 구해진다.
    outputs, ys = np.hsplit(output, self.cnts), np.hsplit(y, self.cnts)   #np.hsplit()를 이용해 미니배치 데이터 각 행에 당긴 데이터별 신경망 처리 결과를 열 기준으로 분할한다.
    
    loss0, aux0 = self.base.forward_postproc(outputs[0], ys[0], 'select')
    loss1, aux1 = self.base.forward_postproc(outputs[1], ys[1], 'select')
        
    return loss0 + loss1, [aux0, aux1]

In [5]:
#복합 출력 처리를 위한 역전파 함수 정의
def office31_backprop_postproc(self, G_loss, aux):
    aux0, aux1 = aux
    
    G_ouput0 = self.base.backprop_postproc(G_loss, aux0, 'select')  #도메인 정보에 대한 손실 기울기
    G_ouput1 = self.base.backprop_postproc(G_loss, aux1, 'select')  #품목 정보에 대한 손실 기울기
    
    return np.hstack([G_ouput0, G_ouput1])

In [6]:
#정확도 계산 함수와 로그 출력 함수 정의
def office31_eval_accuracy(self, x, y, output):
    outputs, ys = np.hsplit(output, self.cnts), np.hsplit(y, self.cnts)   #output과 ys를 두 부분으로 분할

    acc0 = self.base.eval_accuracy(x, ys[0], outputs[0], 'select')      #기반 클래스의 정확도 계산 함수를 호출하여 각각에 대한 정확도
    acc1 = self.base.eval_accuracy(x, ys[1], outputs[1], 'select')      #acc0과 acc1을 얻어낼 수 있다.
    
    return [acc0, acc1]       #도메인과 품목 선택에 대한 정확도 쌍을 반환한다.

def office31_train_prt_result(self, epoch, costs, accs, acc, time1, time2):
    acc_pair = np.mean(accs, axis=0)
    print('    Epoch {}: cost={:5.3f}, ' \
          'accuracy={:5.3f}+{:5.3f}/{:5.3f}+{:5.3f} ({}/{} secs)'. \
          format(epoch, np.mean(costs), acc_pair[0], acc_pair[1], \
                 acc[0], acc[1], time1, time2))

def office31_test_prt_result(self, name, acc, time):
    print('Model {} test report: accuracy = {:5.3f}+{:5.3f}, ({} secs)\n'. \
          format(name, acc[0], acc[1], time))

In [7]:
#시각화 관련 함수 정의
def office31_get_estimate(self, output):
    outputs = np.hsplit(output, self.cnts)

    estimate0 = self.base.get_estimate(outputs[0], 'select')  #분할된 두 부분에 대해 기반 클래스의 추정치 산출 메서드를 호출한다.
    estimate1 = self.base.get_estimate(outputs[1], 'select')
    
    return np.hstack([estimate0, estimate1])   #np.hstack()로 이들을 연접시켜 반환한다.

def office31_visualize(self, xs, estimates, answers):
    draw_images_horz(xs, self.image_shape)     #시각화 대상 데이터들의 이미지를 출력
    
    ests, anss = np.hsplit(estimates, self.cnts), np.hsplit(answers, self.cnts)    #np.hsplit()을 이용해 추정 정보와 정답 정보 부분을 분할
    
    captions = ['도메인', '상품']
    
    for m in range(2):    #두 번 반복 실행해 처음에는 도메인 분류에 대해, 두 번째는 상품 분류에 대해 show_select_results()함수가 호출되어 선택 내용을 출력
        print('[ {} 추정결과 ]'.format(captions[m]))
        show_select_results(ests[m], anss[m], self.target_names[m], 8)

In [8]:
#메서드 일괄 등록
Office31Dataset.__init__          = office31_init
Office31Dataset.forward_postproc  = office31_forward_postproc
Office31Dataset.backprop_postproc = office31_backprop_postproc
Office31Dataset.eval_accuracy     = office31_eval_accuracy
Office31Dataset.get_estimate      = office31_get_estimate
Office31Dataset.train_prt_result  = office31_train_prt_result
Office31Dataset.test_prt_result   = office31_test_prt_result
Office31Dataset.visualize         = office31_visualize